In [1]:
import sys
import nltk
import pandas as pd
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models
from gensim.models import TfidfModel
import json
import ldaconfig
sys.path.append('../../') 
from utils import Utils
utils_obj = Utils()

/home/sanket/.local/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [5]:
df = utils_obj.load_data("../../Dataset/dataset.csv")
documents=df['documents'].tolist()
processed_documents = utils_obj.preprocess_docs(documents)
processed_documents_words = utils_obj.tokenize(processed_documents)

In [6]:
print(processed_documents[1234])

good price quick delivery good choice fitter


In [7]:
bigrams_phrases = gensim.models.Phrases(processed_documents_words, min_count=5, threshold=50)
trigrams_phrases = gensim.models.Phrases(bigrams_phrases[processed_documents_words], min_count=5, threshold=50)

bigram = gensim.models.phrases.Phraser(bigrams_phrases)
trigram = gensim.models.phrases.Phraser(trigrams_phrases)

def create_bigrams(texts):
    return ([bigram[doc] for doc in texts])

def create_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = create_bigrams(processed_documents_words)
data_bigrams_trigrams = create_trigrams(data_bigrams)

print(data_bigrams_trigrams[121])

['seamless', 'hasslefree', 'experience', 'seamless', 'delivery', 'fitting', 'payment', 'good', 'price', 'able', 'select', 'tyre', 'want', 'advance', 'good', 'range', 'suit_budget', 'able', 'schedule', 'timeday', 'suit', 'handy']


In [8]:
id2word = corpora.Dictionary(data_bigrams_trigrams)
texts = data_bigrams_trigrams
corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words=[]
words_missing_in_tfidf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = []
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]
    
    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

In [2]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=ldaconfig.NUM_TOPICS,
                                           random_state=1,
                                           chunksize=100,
                                           passes=20,
                                           alpha="auto")

NameError: name 'corpus' is not defined

In [79]:
topics = lda_model.show_topics(num_words=25, formatted=False, num_topics=15)

In [55]:
topics_dict={}
for topic in topics:
    topics_dict[str(topic[0])]=str(topic[1])

with open("lda_topics.json", "w") as outfile:
    json.dump(topics_dict, outfile)

/home/sanket/.local/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/sanket/.local/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/sanket/.local/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/sanket/.local/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/sanket/.local/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp mo

In [10]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

/home/sanket/.local/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14     0.521416  0.046120       1        1  27.375733
9      0.042187 -0.507623       2        1  18.762461
5     -0.430117 -0.261476       3        1  10.856582
4     -0.247403  0.418398       4        1   7.215207
11     0.241665 -0.344354       5        1   6.724506
1      0.374374 -0.173454       6        1   5.961631
10     0.026288  0.436845       7        1   4.792949
2     -0.170707 -0.340426       8        1   3.668583
7     -0.427185  0.129981       9        1   3.589468
12     0.233959  0.350176      10        1   3.337474
6      0.262479  0.147876      11        1   2.440506
8      0.020681  0.088430      12        1   1.664283
0     -0.265128 -0.044929      13        1   1.472406
13    -0.168688  0.159561      14        1   1.079367
3     -0.013823 -0.105126      15        1   1.058844, topic_info=          Term         Freq        Total Category  logprob  loglift
21        tyre  8434.000000  8434.000000  Default  30.0000  30.0000
20     service  5687.000000  5687.000000  Default  29.0000  29.0000
14        easy  2815.000000  2815.000000  Default  28.0000  28.0000
22        good  4094.000000  4094.000000  Default  27.0000  27.0000
29       price  3855.000000  3855.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
922     making     0.058547     1.103494  Topic15  -9.9536   1.6116
613     people     0.058610     1.117714  Topic15  -9.9526   1.5999
1272       mot     0.058589     1.117075  Topic15  -9.9529   1.6001
301   expected     0.058567     1.119411  Topic15  -9.9533   1.5976
383     budget     0.058544     1.113078  Topic15  -9.9537   1.6029

[614 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
362       1  0.997515         able
488       7  0.989998       agreed
47        5  0.995931         also
948       7  0.981321  alternative
94        2  0.997949       always
...     ...       ...          ...
425       3  0.996093        wheel
228       7  0.992171        whole
218       5  0.997696         work
426       3  0.991282        wrong
378       6  0.991447         year

[275 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 10, 6, 5, 12, 2, 11, 3, 8, 13, 7, 9, 1, 14, 4])

/home/sanket/.local/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/sanket/.local/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/sanket/.local/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/sanket/.local/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/sanket/.local/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp mo